## Immigration data. DE Capstone Project

### Project Summary
We need to monitor immigration traffic among the cities. We may want to analize the dynamics of the population on the level of any state or city.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd

In [3]:
pd.options.display.max_columns = 30
pd.options.display.max_rows = 100

## Step 1: Scope the Project and Gather Data

### Datasets

* **I94 Immigration Data**: 
This data comes from the US National Tourism and Trade Office. A data dictionary is described below. [This is where the data comes from](https://travel.trade.gov/research/reports/i94/historical/2016.html).

* **World Temperature Data**: This dataset came from Kaggle. You can read more about it [here](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data).
* **U.S. City Demographic Data**: This data comes from OpenSoft. You can read more about it [here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/).

* **Airport Code Table**: This is a simple table of airport codes and corresponding cities. It comes from [here](https://datahub.io/core/airport-codes#data).

In [4]:
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.\
# config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
# .enableHiveSupport().getOrCreate()
# df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [5]:
# #write to parquet
# df_spark.write.parquet("sas_data")
# df_spark=spark.read.parquet("sas_data")

The data was early put in S3 bucket `dend-veegaaa-capstone`

## Step 2: Explore and Assess the Data

In [6]:
!ls

airport-codes_csv.csv		 immigration_data_sample.csv
Capstone Project.ipynb		 sas_data
Exploring data.ipynb		 us-cities-demographics.csv
I94_SAS_Labels_Descriptions.SAS


### Explore the Data 

In [7]:
df_airport_codes = pd.read_csv('airport-codes_csv.csv')
df_airport_codes.shape
df_airport_codes.head()

(55075, 12)

ident           type                                name  elevation_ft  \
0   00A       heliport                   Total Rf Heliport          11.0   
1  00AA  small_airport                Aero B Ranch Airport        3435.0   
2  00AK  small_airport                        Lowell Field         450.0   
3  00AL  small_airport                        Epps Airpark         820.0   
4  00AR         closed  Newport Hospital & Clinic Heliport         237.0   

  continent iso_country iso_region  municipality gps_code iata_code  \
0       NaN          US      US-PA      Bensalem      00A       NaN   
1       NaN          US      US-KS         Leoti     00AA       NaN   
2       NaN          US      US-AK  Anchor Point     00AK       NaN   
3       NaN          US      US-AL       Harvest     00AL       NaN   
4       NaN          US      US-AR       Newport      NaN       NaN   

  local_code                            coordinates  
0        00A     -74.93360137939453, 40.07080078125  
1       00AA                 -101.473911, 38.704022  
2       00AK            -151.695999146, 59.94919968  
3       00AL  -86.77030181884766, 34.86479949951172  
4        NaN                    -91.254898, 35.6087

In [8]:
df_demographics = pd.read_csv('us-cities-demographics.csv', sep=';')
df_demographics.shape
df_demographics.head()

(2891, 12)

City          State  Median Age  Male Population  \
0     Silver Spring       Maryland        33.8          40601.0   
1            Quincy  Massachusetts        41.0          44129.0   
2            Hoover        Alabama        38.5          38040.0   
3  Rancho Cucamonga     California        34.5          88127.0   
4            Newark     New Jersey        34.6         138040.0   

   Female Population  Total Population  Number of Veterans  Foreign-born  \
0            41862.0             82463              1562.0       30908.0   
1            49500.0             93629              4147.0       32935.0   
2            46799.0             84839              4819.0        8229.0   
3            87105.0            175232              5821.0       33878.0   
4           143873.0            281913              5829.0       86253.0   

   Average Household Size State Code                       Race  Count  
0                    2.60         MD         Hispanic or Latino  25924  
1                    2.39         MA                      White  58723  
2                    2.58         AL                      Asian   4759  
3                    3.18         CA  Black or African-American  24437  
4                    2.73         NJ                      White  76402

In [9]:
df_demographics[df_demographics['City'].str.contains('nolu')]

City   State  Median Age  Male Population  Female Population  \
296   Urban Honolulu  Hawaii        41.4         176807.0           175959.0   
798   Urban Honolulu  Hawaii        41.4         176807.0           175959.0   
1097  Urban Honolulu  Hawaii        41.4         176807.0           175959.0   
1545  Urban Honolulu  Hawaii        41.4         176807.0           175959.0   
1546  Urban Honolulu  Hawaii        41.4         176807.0           175959.0   

      Total Population  Number of Veterans  Foreign-born  \
296             352766             23213.0      101312.0   
798             352766             23213.0      101312.0   
1097            352766             23213.0      101312.0   
1545            352766             23213.0      101312.0   
1546            352766             23213.0      101312.0   

      Average Household Size State Code                               Race  \
296                     2.69         HI                              Asian   
798                     2.69         HI          Black or African-American   
1097                    2.69         HI                 Hispanic or Latino   
1545                    2.69         HI                              White   
1546                    2.69         HI  American Indian and Alaska Native   

       Count  
296   240978  
798    11781  
1097   24586  
1545  110508  
1546    5592

In [10]:
df_immigrat = pd.read_csv('immigration_data_sample.csv', index_col=0)
df_immigrat.shape
df_immigrat.head()

(1000, 28)

cicid   i94yr  i94mon  i94cit  i94res i94port  arrdate  i94mode  \
2027561  4084316.0  2016.0     4.0   209.0   209.0     HHW  20566.0      1.0   
2171295  4422636.0  2016.0     4.0   582.0   582.0     MCA  20567.0      1.0   
589494   1195600.0  2016.0     4.0   148.0   112.0     OGG  20551.0      1.0   
2631158  5291768.0  2016.0     4.0   297.0   297.0     LOS  20572.0      1.0   
3032257   985523.0  2016.0     4.0   111.0   111.0     CHM  20550.0      3.0   

        i94addr  depdate  i94bir  i94visa  count  dtadfile visapost occup  \
2027561      HI  20573.0    61.0      2.0    1.0  20160422      NaN   NaN   
2171295      TX  20568.0    26.0      2.0    1.0  20160423      MTR   NaN   
589494       FL  20571.0    76.0      2.0    1.0  20160407      NaN   NaN   
2631158      CA  20581.0    25.0      2.0    1.0  20160428      DOH   NaN   
3032257      NY  20553.0    19.0      2.0    1.0  20160406      NaN   NaN   

        entdepa entdepd  entdepu matflag  biryear   dtaddto gender  insnum  \
2027561       G       O      NaN       M   1955.0  07202016      F     NaN   
2171295       G       R      NaN       M   1990.0  10222016      M     NaN   
589494        G       O      NaN       M   1940.0  07052016      M     NaN   
2631158       G       O      NaN       M   1991.0  10272016      M     NaN   
3032257       Z       K      NaN       M   1997.0  07042016      F     NaN   

        airline        admnum  fltno visatype  
2027561      JL  5.658267e+10  00782       WT  
2171295     *GA  9.436200e+10  XBLNG       B2  
589494       LH  5.578047e+10  00464       WT  
2631158      QR  9.478970e+10  00739       B2  
3032257     NaN  4.232257e+10   LAND       WT

In [11]:
import pyarrow.parquet as pq

dataset = pq.ParquetDataset('sas_data')
dataset.schema

required group field_id=0 spark_schema {
  optional double field_id=1 cicid;
  optional double field_id=2 i94yr;
  optional double field_id=3 i94mon;
  optional double field_id=4 i94cit;
  optional double field_id=5 i94res;
  optional binary field_id=6 i94port (String);
  optional double field_id=7 arrdate;
  optional double field_id=8 i94mode;
  optional binary field_id=9 i94addr (String);
  optional double field_id=10 depdate;
  optional double field_id=11 i94bir;
  optional double field_id=12 i94visa;
  optional double field_id=13 count;
  optional binary field_id=14 dtadfile (String);
  optional binary field_id=15 visapost (String);
  optional binary field_id=16 occup (String);
  optional binary field_id=17 entdepa (String);
  optional binary field_id=18 entdepd (String);
  optional binary field_id=19 entdepu (String);
  optional binary field_id=20 matflag (String);
  optional double field_id=21 biryear;
  optional binary field_id=22 dtaddto (String);
  optional binary field_id=23 

In [12]:
table = dataset.read(columns=['cicid', 'i94yr'])
table

pyarrow.Table
cicid: double
i94yr: double

* Demographics dataset is currently aggregated on the level of particular city

### Data from S3

In [37]:
KEY=""
SECRET=""

In [36]:
import boto3

s3 = boto3.resource('s3',
                    region_name="us-west-2",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                   )

sampleDbBucket =  s3.Bucket("dend-veegaaa-capstone")
# obj_list = list(sampleDbBucket.objects.filter(Prefix='song_data/A/B/C/'))
obj_list = list(sampleDbBucket.objects.filter())
obj = obj_list[0]
obj

s3.ObjectSummary(bucket_name='dend-veegaaa-capstone', key='immigration_data_sample.csv')

## Step 3: Define the Data Model

### Conceptual Data Model

The data model must serve the followeing needs:
* Analytics team are able to monitor immigration traffic according to logs
* One can aggregate total population on city and state levels

Therefore, it's appropriate to create a star model with the followeing structure:

**Fact table**

_immigration_facts_
* *cicid* - id
* *i94port* - City code
* *i94addr* - State code
* *dtadfile* - Date added to I-94 Files
* *dtaddto* - Date to which admitted to U.S. (allowed to stay until)
* *admnum* - Admission Number

**Dimension tables**

_states_
* *state_code* - state code
* *state_name* - state name

_cities_
* *city_name* - city name
* *state_code* - state code
* *total_pop* - total population

_times_
* *date* - date
* *day* - day
* *month* - month
* *year* - year
* *weekday* - weekday


In [17]:
# for col in df_immigrat.columns:
#     print(f"{col} text,")
for col in df_demographics.columns:
    print(f"{str.lower(col)} text,")

city text,
state text,
median age text,
male population text,
female population text,
total population text,
number of veterans text,
foreign-born text,
average household size text,
state code text,
race text,
count text,


### Inserts

### Mapping Out Data Pipelines
The steps necessary to pipeline the data into the chosen data model

1. Create tables
2. Copy data to staging tables
3. Perform ETL to insert the data into Fact & Dimension tables

In [18]:
query_copy = (f"""
copy staging_demographics 
from 's3://dend-veegaaa-capstone/us-cities-demographics.csv' 
ACCESS_KEY_ID '{KEY}'
SECRET_ACCESS_KEY '{SECRET}'
IGNOREHEADER 1
DELIMITER ';'
;""")

In [19]:
cur.execute(query_copy)
conn.commit()

In [20]:
query_copy = (f"""
copy staging_immigration 
from 's3://dend-veegaaa-capstone/immigration_data_sample.csv' 
ACCESS_KEY_ID '{KEY}'
SECRET_ACCESS_KEY '{SECRET}'
IGNOREHEADER 1
DELIMITER ','
;""")

In [21]:
cur.execute(query_copy)
conn.commit()

In [22]:
# conn.rollback()

## Step 4: Run Pipelines to Model the Data 

### Create the data model
Build the data pipelines to create the data model.

In [23]:
import psycopg2

In [24]:
conn = psycopg2.connect(f"host=redshift-cluster-1.cmv39mltiyk7.us-west-2.redshift.amazonaws.com \
    dbname=dev \
    user=awsuser \
    password=I4lqx2jp \
    port=5439")

In [25]:
cur = conn.cursor()

In [26]:
# DROP TABLES
# table_drop = "drop table IF EXISTS staging_immigration"
table_list = ['staging_immigration', 'staging_demographics', 'immigration_facts', 'times', 'states', 'cities']

In [27]:
# CREATE TABLES

staging_events_table_create= ("""
CREATE TABLE IF NOT EXISTS staging_immigration
(
index_col text,
cicid text,
i94yr text,
i94mon text,
i94cit text,
i94res text,
i94port text,
arrdate text,
i94mode text,
i94addr text,
depdate text,
i94bir text,
i94visa text,
count text,
dtadfile text,
visapost text,
occup text,
entdepa text,
entdepd text,
entdepu text,
matflag text,
biryear text,
dtaddto text,
gender text,
insnum text,
airline text,
admnum text,
fltno text,
visatype text
);
""")

staging_demogr_table_create =  ("""
CREATE TABLE IF NOT EXISTS staging_demographics
(
city text,
state text,
median_age text,
male_population text,
female_population text,
total_population text,
number_of_veterans text,
foreign_born text,
average_household_size text,
state text,
race text,
count text
);
""")

In [28]:
immigration_facts_table_create = ("""
CREATE TABLE IF NOT EXISTS immigration_facts
(
cicid BIGINT PRIMARY KEY,
i94port text,
i94addr text,
dtadfile date,
dtaddto date,
admnum BIGINT
);
""")

states_table_create = ("""
CREATE TABLE IF NOT EXISTS states
(
state_code text PRIMARY KEY,
state_name text
);
""")

cities_table_create = ("""
CREATE TABLE IF NOT EXISTS cities
(
city_name text,
state_code text,
total_pop real,
PRIMARY KEY(city_name, state_code)
);
""")

time_table_create = ("""
CREATE TABLE IF NOT EXISTS times
(
date timestamp NOT NULL PRIMARY KEY,
day int,
month int,
year int,
weekday int
);
""")

In [29]:
cur.execute(staging_events_table_create)
cur.execute(staging_demogr_table_create)
cur.execute(immigration_facts_table_create)
cur.execute(states_table_create)
cur.execute(cities_table_create)
cur.execute(time_table_create)

In [30]:
conn.commit()

In [31]:
insert_queries = [
("""
INSERT INTO immigration_facts(cicid,i94port,i94addr,dtadfile,dtaddto,admnum)
    SELECT 
        cast(cast(cicid as real) as bigint) cicid,
        i94port,
        i94addr,
        to_date(dtadfile, 'yyyymmdd') as dtadfile,
        case when dtaddto <> 'D/S' then to_date(dtaddto, 'mmddyyyy') else null end as dtaddto,
        cast(cast(admnum as real) as bigint) admnum
    FROM staging_immigration
"""),
("""
INSERT INTO states(state_code, state_name)
    SELECT distinct
        state_code,
        state state_name
    FROM staging_demographics
"""),
("""
INSERT INTO cities(city_name, state_code, total_pop)
    SELECT distinct
        city city_name,
        state_code,
        cast(cast(total_population as real) as bigint) total_pop
    FROM staging_demographics
"""),
("""
INSERT INTO times(date,day,month,year,weekday)
    SELECT distinct * from (
        SELECT
            dtadfile as date,
            extract(day from dtadfile) as day,
            extract(month from dtadfile) as month,
            extract(year from dtadfile) as year,
            extract(dayofweek from dtadfile) as weekday
        FROM immigration_facts
        UNION ALL
        SELECT
            dtaddto as date,
            extract(day from dtaddto) as day,
            extract(month from dtaddto) as month,
            extract(year from dtaddto) as year,
            extract(dayofweek from dtaddto) as weekday
        FROM immigration_facts
        ) t
        where date is not null
"""),
]

In [32]:
for query in insert_queries:
    cur.execute(query)

In [33]:
# conn.rollback()

In [34]:
conn.commit()

### Data Quality Checks

#### All data quality checks are performing within Airflow DAG

### Data dictionary 

**Fact table**

_immigration_facts_
* *cicid* - id
* *i94port* - City code
* *i94addr* - State code
* *dtadfile* - Date added to I-94 Files
* *dtaddto* - Date to which admitted to U.S. (allowed to stay until)
* *admnum* - Admission Number

**Dimension tables**

_states_
* *state_code* - state code
* *state_name* - state name

_cities_
* *city_name* - city name
* *state_code* - state code
* *total_pop* - total population

_times_
* *date* - date
* *day* - day
* *month* - month
* *year* - year
* *weekday* - weekday

## Step 5: Project Write Up

###### Rationale for the choice of tools and technologies

Once we get the data from a source, wheter it's a local filesystem or S3, we need to perform ELT process to prepare it for the further analytics. That's why it's decided to store it in Redshift - cloud data warehouse with support of running sql-like queries.

###### Possible scenarios and logic to handle it.
**1. The data was increased by 100x.**
Redshift is a scalable warehouse and the ELT process is performed in batch manner so this case wouldn't become a bottleneck.

**2. The data populates a dashboard that must be updated on a daily basis by 7am every day.**
The whole process is scheduled with Airflow DAG which can be set up to run on any interval.

**3. The database needed to be accessed by 100+ people.**
Again, Redshift can be scaled up and allow access to the data for a team of any size.